<a href="https://colab.research.google.com/github/ys2064/NLP_based_NewsAudioSummarizer/blob/main/Training_and_Testing_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Training the BART model

In [ ]:
import os
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import gc
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')
train_folders = [
    '/content/drive/MyDrive/train_files_set1/',
    '/content/drive/MyDrive/train_files_set2/'
]
summary_folders = [
    '/content/drive/MyDrive/reference summary for train set1/',
    '/content/drive/MyDrive/reference summary for train set2/'
]
output_summary_folder = '/content/drive/MyDrive/NLP project_BART_generated_summaries/'

os.makedirs(output_summary_folder, exist_ok=True)
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
def load_and_preprocess_data(train_folders, summary_folders):
    train_texts = []
    summary_texts = []
    for train_folder, summary_folder in zip(train_folders, summary_folders):
        for filename in os.listdir(train_folder):
            if filename.endswith(".txt"):
                with open(os.path.join(train_folder, filename), 'r') as f:
                    train_texts.append(f.read())
                summary_filename = 'summary_' + filename
                summary_path = os.path.join(summary_folder, summary_filename)
                if os.path.exists(summary_path):
                    with open(summary_path, 'r') as f:
                        summary_texts.append(f.read())
                else:
                    print(f"Warning: No matching summary found for {filename}")
    return train_texts, summary_texts
train_texts, summary_texts = load_and_preprocess_data(train_folders, summary_folders)
inputs = tokenizer(train_texts, padding=True, truncation=True, max_length=1024, return_tensors="pt")
labels = tokenizer(summary_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

train_dataset = Dataset.from_dict({
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask'],
    'labels': labels['input_ids']
})
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/BART_summary_output',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_strategy="steps",
    logging_dir='/content/drive/MyDrive/BART_logs_two',
    logging_steps=50,
    save_total_limit=1,
    report_to="none",
    eval_strategy="no",
    save_steps=500,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)
trainer.train()
model.save_pretrained('/content/drive/MyDrive/BART_trained_model')
tokenizer.save_pretrained('/content/drive/MyDrive/BART_trained_model')
def generate_summaries(train_folders, summary_folders, output_summary_folder):
    for train_folder, summary_folder in zip(train_folders, summary_folders):
        for filename in os.listdir(train_folder):
            if filename.endswith(".txt"):
                with open(os.path.join(train_folder, filename), 'r') as f:
                    text = f.read()
                inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024).to(device)
                summary_ids = model.generate(
                    inputs['input_ids'],
                    max_length=350,
                    min_length=200,
                    num_beams=4,
                    early_stopping=True
                )
                summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
                output_path = os.path.join(output_summary_folder, 'generated_' + filename)
                with open(output_path, 'w') as f:
                    f.write(summary)
                print(f"Generated summary saved for {filename}")
generate_summaries(train_folders, summary_folders, output_summary_folder)
gc.collect()
print("Training and summary generation completed.")

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Step,Training Loss
50,2.342000
100,1.302100
150,1.046000
200,0.776900


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Generated summary saved for Iran’s missile attack on Israel, Helene’s political fallout and fentanyl’s unlikely route.wav.txt
Generated summary saved for Harris bashes Trump, Democrat VP dilemma and Palestinian unity deal.txt
Generated summary saved for Israel sends more ground troops into Lebanon as first combat deaths reported ｜ DW News.txt
Generated summary saved for Hurricane Milton, Mexico mayor’s murder and US election certifiers.txt
Generated summary saved for Israel_s airstrikes hit Beirut as it aims to cripple Hezbollah ｜ DW News.txt
Generated summary saved for Jordan’s balancing act, Google’s court date and Black voters in Georgia.txt
Generated summary saved for Israeli bombardment, Hezbollah_s future, US port strike and Austria far-right win.wav.txt
Generated summary saved for Intel puts German plant plans on hold I DW News.txt
Generated summary saved for Hurricane Milton evacuation, Afghan arrested for plotting Election Day attack & Hezbollah latest.txt
Generated summary sa

# Validation set

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
import gc
import os
device = 'cpu'
print(f'Using device: {device}')
val_folders = [
    '/content/drive/MyDrive/val_files_set1/',
    '/content/drive/MyDrive/val_files_set2/'
]
output_summary_folder = '/content/drive/MyDrive/NLP project/BART_generated_summaries_validation/'
os.makedirs(output_summary_folder, exist_ok=True)
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/BART_trained_model').to(device)
max_input_length = 1024
max_summary_length = 350
def split_text_into_chunks(text, max_length):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    return chunks
def generate_validation_summaries(val_folders, output_summary_folder):
    for val_folder in val_folders:
        for filename in os.listdir(val_folder):
            if filename.endswith(".txt"):
                file_path = os.path.join(val_folder, filename)
                with open(file_path, 'r') as f:
                    text = f.read()
                if len(text.strip()) == 0:
                    print(f"Skipping empty input file: {filename}")
                    continue
                model.to('cpu')
                text_chunks = split_text_into_chunks(text, max_input_length)
                full_summary = ""
                for chunk in text_chunks:
                    chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
                    inputs = tokenizer(chunk_text, return_tensors="pt", truncation=True, max_length=max_input_length).to('cpu')
                    if inputs['input_ids'].size(1) == 0:
                        print(f"Skipping file with empty tokenization: {filename}")
                        continue
                    summary_ids = model.generate(
                        inputs['input_ids'],
                        max_length=max_summary_length,
                        min_length=200,
                        num_beams=3,
                        early_stopping=True
                    )
                    if summary_ids.size(1) == 0:
                        print(f"Summary generation failed for {filename}")
                        continue
                    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
                    if len(summary.strip()) == 0:
                        print(f"Generated empty summary for {filename}, skipping.")
                        continue
                    full_summary += summary + " "
                if len(full_summary.strip()) == 0:
                    print(f"Generated empty summary for {filename}, skipping.")
                    continue
                output_path = os.path.join(output_summary_folder, f'generated_val_{filename}')
                with open(output_path, 'w') as f:
                    f.write(full_summary)
                print(f"Generated summary saved for {filename}")
generate_validation_summaries(val_folders, output_summary_folder)
gc.collect()
print("Validation summary generation completed.")

Using device: cpu
Generated summary saved for Europe floods update： What_s the situation on the ground？ ｜ DW News.txt
Generated summary saved for Freed Yazidi captive, Lebanon’s Syria crossing, early US concerns on Gaza and Houthi emails.txt
Generated summary saved for French election results in surprise win for the left, Biden meme and could Harris could beat Trump？.txt
Generated summary saved for Harris v Trump debate, undecided voters, Taylor Swift’s endorsement and Russia’s Iranian missiles.txt
Generated summary saved for Gershkovich released in prisoner swap, Hezbollah warns Israel and Olympic boxing controversy.txt
Generated summary saved for Floods in central Europe claim several lives as water levels continue to rise ｜ DW News.txt
Generated summary saved for Everything you need to know about the presidential debate.txt
Generated summary saved for Handover to Harris, Biden_s legacy and Gaza ceasefire plan.txt
Generated summary saved for Germany expands border controls to curb mi

In [ ]:
pip install nltk rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0ea526b0bdd6e72f6776959501ee716445a18cb2634362d539082f9c890db04f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import os
import nltk
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)
reference_summary_folders = [
    '/content/drive/MyDrive/reference summary for val set1/',
    '/content/drive/MyDrive/reference summary for val set2/'
]
generated_summary_folder = '/content/drive/MyDrive/NLP project/BART_generated_summaries_validation/'
rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
def calculate_rouge_meteor(reference_summary_folders, generated_summary_folder):
    rouge_scores = {"rouge1": {"precision": [], "recall": [], "f1": []},
                    "rouge2": {"precision": [], "recall": [], "f1": []},
                    "rougeL": {"precision": [], "recall": [], "f1": []}}
    meteor_scores = []
    for reference_folder in reference_summary_folders:
        for ref_filename in os.listdir(reference_folder):
            if ref_filename.endswith(".txt"):
                stripped_ref_filename = ref_filename.replace("summary_", "")
                gen_filename = f"generated_val_{stripped_ref_filename}"
                ref_file_path = os.path.join(reference_folder, ref_filename)
                gen_file_path = os.path.join(generated_summary_folder, gen_filename)
                if not os.path.exists(gen_file_path):
                    continue
                with open(ref_file_path, 'r', encoding='utf-8-sig') as ref_file:
                    reference_summary = ref_file.read().strip()
                with open(gen_file_path, 'r', encoding='utf-8-sig') as gen_file:
                    generated_summary = gen_file.read().strip()
                if not reference_summary or not generated_summary:
                    continue
                rouge_result = rouge_scorer_instance.score(reference_summary, generated_summary)
                for metric in ["rouge1", "rouge2", "rougeL"]:
                    rouge_scores[metric]["precision"].append(rouge_result[metric].precision)
                    rouge_scores[metric]["recall"].append(rouge_result[metric].recall)
                    rouge_scores[metric]["f1"].append(rouge_result[metric].fmeasure)
                reference_tokens_meteor = nltk.word_tokenize(reference_summary.lower())
                generated_tokens_meteor = nltk.word_tokenize(generated_summary.lower())
                meteor_score_value = meteor_score([reference_tokens_meteor], generated_tokens_meteor)
                meteor_scores.append(meteor_score_value)
    def average_scores(scores_dict):
        return {metric: sum(scores_dict[metric]) / len(scores_dict[metric]) if scores_dict[metric] else 0
                for metric in scores_dict}
    average_rouge_scores = {metric: average_scores(rouge_scores[metric]) for metric in rouge_scores}
    average_meteor = sum(meteor_scores) / len(meteor_scores) if meteor_scores else 0
    print(f"ROUGE-1 Precision: {average_rouge_scores['rouge1']['precision']:.2f}")
    print(f"ROUGE-1 Recall: {average_rouge_scores['rouge1']['recall']:.2f}")
    print(f"ROUGE-1 F1: {average_rouge_scores['rouge1']['f1']:.2f}")
    print("-" * 30)
    print(f"ROUGE-2 Precision: {average_rouge_scores['rouge2']['precision']:.2f}")
    print(f"ROUGE-2 Recall: {average_rouge_scores['rouge2']['recall']:.2f}")
    print(f"ROUGE-2 F1: {average_rouge_scores['rouge2']['f1']:.2f}")
    print("-" * 30)
    print(f"ROUGE-L Precision: {average_rouge_scores['rougeL']['precision']:.2f}")
    print(f"ROUGE-L Recall: {average_rouge_scores['rougeL']['recall']:.2f}")
    print(f"ROUGE-L F1: {average_rouge_scores['rougeL']['f1']:.2f}")
    print("-" * 30)
    print(f"METEOR Score: {average_meteor:.2f}")
calculate_rouge_meteor(reference_summary_folders, generated_summary_folder)


ROUGE-1 Precision: 0.37
ROUGE-1 Recall: 0.59
ROUGE-1 F1: 0.44
------------------------------
ROUGE-2 Precision: 0.13
ROUGE-2 Recall: 0.20
ROUGE-2 F1: 0.15
------------------------------
ROUGE-L Precision: 0.18
ROUGE-L Recall: 0.29
ROUGE-L F1: 0.22
------------------------------
METEOR Score: 0.38


# Testing

In [ ]:
import os
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
import gc
device = 'cpu'
print(f'Using device: {device}')
test_folders = [
    '/content/drive/MyDrive/test_files_set1/','/content/drive/MyDrive/test_files_set2/'
]
output_summary_folder = '/content/drive/MyDrive/NLP project/BART_generated_summaries_test/'
os.makedirs(output_summary_folder, exist_ok=True)
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/BART_trained_model').to(device)
max_input_length = 1024
max_summary_length = 350
def split_text_into_chunks(text, max_length):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    return chunks
def generate_test_summaries(test_folders, output_summary_folder):
    for test_folder in test_folders:
        for filename in os.listdir(test_folder):
            if filename.endswith(".txt"):
                file_path = os.path.join(test_folder, filename)
                with open(file_path, 'r') as f:
                    text = f.read()
                if len(text.strip()) == 0:
                    print(f"Skipping empty input file: {filename}")
                    continue
                model.to('cpu')
                text_chunks = split_text_into_chunks(text, max_input_length)
                full_summary = ""
                for chunk in text_chunks:
                    chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
                    inputs = tokenizer(chunk_text, return_tensors="pt", truncation=True, max_length=max_input_length).to('cpu')
                    if inputs['input_ids'].size(1) == 0:
                        print(f"Skipping file with empty tokenization: {filename}")
                        continue
                    summary_ids = model.generate(
                        inputs['input_ids'],
                        max_length=max_summary_length,
                        min_length=200,
                        num_beams=3,
                        early_stopping=True
                    )
                    if summary_ids.size(1) == 0:
                        print(f"Summary generation failed for {filename}")
                        continue
                    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
                    if len(summary.strip()) == 0:
                        print(f"Generated empty summary for {filename}, skipping.")
                        continue
                    full_summary += summary + " "
                if len(full_summary.strip()) == 0:
                    print(f"Generated empty summary for {filename}, skipping.")
                    continue
                output_path = os.path.join(output_summary_folder, f'generated_test_{filename}')
                with open(output_path, 'w') as f:
                    f.write(full_summary)
                print(f"Generated summary saved for {filename}")
generate_test_summaries(test_folders, output_summary_folder)
gc.collect()
print("Test summary generation completed.")

Using device: cpu
Generated summary saved for Central Europe in grips of worst flooding in decades ｜ DW News.txt
Generated summary saved for EU_s €300bn clash with China - interview with European Commissioner Jutta Urpilainen  ｜ DW News.txt
Generated summary saved for Claudia Sheinbaum sworn-in as Mexico_s first ever woman president ｜ DW News.txt
Generated summary saved for Central Europe under water ｜ DW News.txt
Generated summary saved for _The game is about to change_ -  Israeli general says Lebanon in war_s new focus ｜ DW News.txt
Generated empty summary for Biden hosts NATO summit, poised to unveil new aid for Ukraine, Modi meets Putin on visit to Russia.txt, skipping.
Generated summary saved for Biden hosts NATO summit, poised to unveil new aid for Ukraine, Modi meets Putin on visit to Russia.txt
Generated summary saved for Coffee and politics at the _traitors club_ in a divided Cyprus.txt
Generated summary saved for _Israel_s incursion into Lebanon, Helene_s devastation and VP d

In [ ]:
import os
import nltk
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)
reference_summary_folders = [
    '/content/drive/MyDrive/reference summary for test set1/',
    '/content/drive/MyDrive/reference summary for test set2/'
]
generated_summary_folder = '/content/drive/MyDrive/NLP project/BART_generated_summaries_test/'
rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
def calculate_rouge_meteor(reference_summary_folders, generated_summary_folder):
    rouge_scores = {"rouge1": {"precision": [], "recall": [], "f1": []},
                    "rouge2": {"precision": [], "recall": [], "f1": []},
                    "rougeL": {"precision": [], "recall": [], "f1": []}}
    meteor_scores = []
    for reference_folder in reference_summary_folders:
        for ref_filename in os.listdir(reference_folder):
            if ref_filename.endswith(".txt"):
                stripped_ref_filename = ref_filename.replace("summary_", "")
                gen_filename = f"generated_test_{stripped_ref_filename}"
                ref_file_path = os.path.join(reference_folder, ref_filename)
                gen_file_path = os.path.join(generated_summary_folder, gen_filename)
                if not os.path.exists(gen_file_path):
                    continue
                with open(ref_file_path, 'r', encoding='utf-8-sig') as ref_file:
                    reference_summary = ref_file.read().strip()
                with open(gen_file_path, 'r', encoding='utf-8-sig') as gen_file:
                    generated_summary = gen_file.read().strip()
                if not reference_summary or not generated_summary:
                    continue
                rouge_result = rouge_scorer_instance.score(reference_summary, generated_summary)
                for metric in ["rouge1", "rouge2", "rougeL"]:
                    rouge_scores[metric]["precision"].append(rouge_result[metric].precision)
                    rouge_scores[metric]["recall"].append(rouge_result[metric].recall)
                    rouge_scores[metric]["f1"].append(rouge_result[metric].fmeasure)
                reference_tokens_meteor = nltk.word_tokenize(reference_summary.lower())
                generated_tokens_meteor = nltk.word_tokenize(generated_summary.lower())
                meteor_score_value = meteor_score([reference_tokens_meteor], generated_tokens_meteor)
                meteor_scores.append(meteor_score_value)
    def average_scores(scores_dict):
        return {metric: sum(scores_dict[metric]) / len(scores_dict[metric]) if scores_dict[metric] else 0
                for metric in scores_dict}
    average_rouge_scores = {metric: average_scores(rouge_scores[metric]) for metric in rouge_scores}
    average_meteor = sum(meteor_scores) / len(meteor_scores) if meteor_scores else 0
    print(f"ROUGE-1 Precision: {average_rouge_scores['rouge1']['precision']:.2f}")
    print(f"ROUGE-1 Recall: {average_rouge_scores['rouge1']['recall']:.2f}")
    print(f"ROUGE-1 F1: {average_rouge_scores['rouge1']['f1']:.2f}")
    print("-" * 30)
    print(f"ROUGE-2 Precision: {average_rouge_scores['rouge2']['precision']:.2f}")
    print(f"ROUGE-2 Recall: {average_rouge_scores['rouge2']['recall']:.2f}")
    print(f"ROUGE-2 F1: {average_rouge_scores['rouge2']['f1']:.2f}")
    print("-" * 30)
    print(f"ROUGE-L Precision: {average_rouge_scores['rougeL']['precision']:.2f}")
    print(f"ROUGE-L Recall: {average_rouge_scores['rougeL']['recall']:.2f}")
    print(f"ROUGE-L F1: {average_rouge_scores['rougeL']['f1']:.2f}")
    print("-" * 30)
    print(f"METEOR Score: {average_meteor:.2f}")
calculate_rouge_meteor(reference_summary_folders, generated_summary_folder)

ROUGE-1 Precision: 0.36
ROUGE-1 Recall: 0.56
ROUGE-1 F1: 0.42
------------------------------
ROUGE-2 Precision: 0.12
ROUGE-2 Recall: 0.19
ROUGE-2 F1: 0.14
------------------------------
ROUGE-L Precision: 0.18
ROUGE-L Recall: 0.28
ROUGE-L F1: 0.21
------------------------------
METEOR Score: 0.36
